In [1]:
rm C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\* -recurse

## To build the project

We really just need to `dotnet restore` and `dotnet build` ...

But in order to ship something, we need to `publish` it.

## Increment the version

I've changed the project files to use the new `--version-suffix` feature:

In [2]:
.{
dotnet restore
dotnet publish -f netcoreapp2.0 -c Release --version-suffix "-beta-4"
dotnet publish -f net462 -c Release --version-suffix "-beta-4"
} | Where { $_ -notmatch "NU1603" }

Restoring packages for C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\PowerShell-Kernel\PowerShell-Kernel.csproj...
  Restoring packages for C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Jupyter\Jupyter.csproj...
  Restore completed in 637.09 ms for C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Jupyter\Jupyter.csproj.
  Restore completed in 2.22 sec for C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\PowerShell-Kernel\PowerShell-Kernel.csproj.
Microsoft (R) Build Engine version 15.3.388.41745 for .NET Core
Copyright (C) Microsoft Corporation. All rights reserved.

  Jupyter -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\netcoreapp2.0\Jupyter.dll
  Jupyter -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\netcoreapp2.0\publish\
  PowerShell-Kernel -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\netcoreapp2.0\PowerShell-Kernel.dll
  PowerShell-Kernel -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release

In [3]:
mkdir Output\Release\Jupyter-PowerShell -force | convert-path | Set-Location
move-item ..\net462\publish PowerShell-Full
move-item ..\netcoreapp2.0\publish PowerShell-Core
copy-item ..\..\..\Install.ps1 .
cd ..
Microsoft.PowerShell.Archive\Compress-Archive Jupyter-PowerShell Jupyter-PowerShell.zip -Force
ls

Directory: C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release


Mode                LastWriteTime         Length Name                                                                   
----                -------------         ------ ----                                                                   
d-----        7/12/2017   1:09 AM                Jupyter-PowerShell                                                     
d-----        7/12/2017   1:09 AM                net462                                                                 
d-----        7/12/2017   1:09 AM                netcoreapp2.0                                                          
-a----        7/12/2017   1:10 AM       17354114 Jupyter-PowerShell.zip